In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import monai
import torch.nn as nn
import torch

from MedSAM_HCP.dataset import MRIDataset, load_datasets
from MedSAM_HCP.MedSAM import MedSAM, medsam_inference
from MedSAM_HCP.build_sam import build_sam_vit_b_multiclass
from MedSAM_HCP.utils_hcp import *
from MedSAM_HCP.loss_funcs_hcp import *
from MedSAM_HCP.distance_map_functions import *

In [4]:
pred = torch.normal(torch.zeros(32,5,256,256), torch.ones(32,5,256,256))
gt = torch.randint(low=0, high=2, size=(32,5,256,256))
gt[:,0,:,:] = 1
class_weights = torch.ones((5))

In [7]:
loss_weighted_ver, _, _, _ = weighted_ce_dice_loss(pred, gt, class_weights, lambda_dice=0.5)
loss_unweighted_ver = unweighted_ce_dice_loss(pred, gt, lambda_dice=0.5)

print(loss_weighted_ver)
print(loss_unweighted_ver)

tensor(1.1759)
tensor(1.1762)


### Test distance map

In [43]:
# generate gt of 2 classes
#gt = np.random.choice(2, size = (1,2,4,4), p=[0.9, 0.1])

gt = np.array([[
    [0, 0, 1, 1, 0, 0],
    [1, 1, 1, 1, 1, 1],
    [1, 1, 0, 0, 1, 1],
    [1, 1, 0, 0, 1, 1],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0]
]]
)
print(gt.shape)


# replace second class with inverse of the first class
#gt[:, 1, :, :] = 1 - gt[:, 0, :, :]


#print(gt[0].shape)

print(gt)

print(np.round(one_hot2dist(gt,dtype=float), 2))

(1, 8, 6)
[[[0 0 1 1 0 0]
  [1 1 1 1 1 1]
  [1 1 0 0 1 1]
  [1 1 0 0 1 1]
  [0 0 1 1 0 0]
  [0 0 1 1 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]]]
[[[ 1.    1.    0.    0.    1.    1.  ]
  [ 0.    0.    0.    0.    0.    0.  ]
  [-1.    0.    1.    1.    0.   -1.  ]
  [ 0.    0.    1.    1.    0.    0.  ]
  [ 1.    1.    0.    0.    1.    1.  ]
  [ 2.    1.    0.    0.    1.    2.  ]
  [ 2.24  1.41  1.    1.    1.41  2.24]
  [ 2.83  2.24  2.    2.    2.24  2.83]]]


In [45]:
# generate gt of 2 classes
#gt = np.random.choice(2, size = (1,2,4,4), p=[0.9, 0.1])

gt = np.array([[
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0]
]]
)
print(gt.shape)


# replace second class with inverse of the first class
#gt[:, 1, :, :] = 1 - gt[:, 0, :, :]


#print(gt[0].shape)

print(gt)

print(np.round(one_hot2dist(gt,dtype=float), 2))

(1, 8, 8)
[[[0 0 0 0 0 0 0 0]
  [0 1 1 1 1 1 1 0]
  [0 1 1 1 1 1 1 0]
  [0 1 1 1 1 1 1 0]
  [0 1 1 1 1 1 1 0]
  [0 1 1 1 1 1 1 0]
  [0 1 1 1 1 1 1 0]
  [0 0 0 0 0 0 0 0]]]
[[[ 1.41  1.    1.    1.    1.    1.    1.    1.41]
  [ 1.    0.    0.    0.    0.    0.    0.    1.  ]
  [ 1.    0.   -1.   -1.   -1.   -1.    0.    1.  ]
  [ 1.    0.   -1.   -2.   -2.   -1.    0.    1.  ]
  [ 1.    0.   -1.   -2.   -2.   -1.    0.    1.  ]
  [ 1.    0.   -1.   -1.   -1.   -1.    0.    1.  ]
  [ 1.    0.    0.    0.    0.    0.    0.    1.  ]
  [ 1.41  1.    1.    1.    1.    1.    1.    1.41]]]
